In [2]:
# !wget https://object.pouta.csc.fi/OPUS-Tanzil/v1/raw/en.zip
# !wget https://object.pouta.csc.fi/OPUS-Tanzil/v1/raw/ms.zip
# !unzip -o en.zip
# !unzip -o ms.zip

In [4]:
# !wget https://object.pouta.csc.fi/OPUS-Tanzil/v1/xml/en-ms.xml.gz
# !gzip -d en-ms.xml.gz

In [12]:
from bs4 import BeautifulSoup
from tqdm import tqdm

with open('en-ms.xml') as fopen:
    xml = fopen.read().split('\n')
    
mapping, ids = {}, []
enable = False
for row in tqdm(xml):
    if '<linkGrp targType' in row:
        s = BeautifulSoup(row)
        r = 'Tanzil/raw/' + s.findAll('linkgrp')[0].get('fromdoc').replace('.gz','')
        l = 'Tanzil/raw/' + s.findAll('linkgrp')[0].get('todoc').replace('.gz','')
        enable = True
    if '<link ' in row and enable:
        s = BeautifulSoup(row)
        t = s.findAll('link')[0].get('xtargets')
        t = list(filter(None, t.split(';')))
        if len(t) != 2:
            continue
        ids.append([t[0].split(), t[1].split()])
    if '</linkGrp>' in row:
        label = f'{l} <> {r}'
        mapping[label] = ids
        enable = False
        ids = []

100%|██████████| 124656/124656 [00:28<00:00, 4336.58it/s]


In [13]:
results = []
for file in tqdm(mapping.keys()):
    l, r = file.split(' <> ')
    ids = mapping[file]
    try:
        with open(l) as fopen:
            left = fopen.read()
        soup_left = BeautifulSoup(left, 'html.parser')
        with open(r) as fopen:
            right = fopen.read()
        soup_right = BeautifulSoup(right, 'html.parser')
        left_dict = {}
        for s in soup_left.find_all('s'):
            i = s.get('id')
            text = s.get_text()
            left_dict[i] = text
        right_dict = {}
        for s in soup_right.find_all('s'):
            i = s.get('id')
            text = s.get_text()
            right_dict[i] = text
        combined = []
        for i in ids:
            try:
                r = ' '.join([right_dict[k] for k in i[0]])
                l = ' '.join([left_dict[k] for k in i[1]])
                combined.append((l, r))
            except:
                pass
        results.extend(combined)
    except Exception as e:
        print(e)
        pass

100%|██████████| 15/15 [00:21<00:00,  1.41s/it]


In [14]:
len(results)

122483

In [15]:
results[:10]

[('Dengan nama Allah, Yang Maha Pemurah, lagi Maha Mengasihani.',
  'In the name of Allah, most benevolent, ever-merciful.'),
 ('Segala puji tertentu bagi Allah, Tuhan yang memelihara dan mentadbirkan sekalian alam.',
  'ALL PRAISE BE to Allah, Lord of all the worlds,'),
 ('Yang Maha Pemurah, lagi Maha Mengasihani.',
  'Most beneficent, ever-merciful,'),
 ('Yang Menguasai pemerintahan hari Pembalasan (hari Akhirat).',
  'King of the Day of Judgement.'),
 ('Engkaulah sahaja (Ya Allah) Yang Kami sembah, dan kepada Engkaulah sahaja kami memohon pertolongan.',
  'You alone we worship, and to You alone turn for help.'),
 ('Tunjukilah kami jalan yang lurus.',
  'Guide us (O Lord) to the path that is straight,'),
 ('Iaitu jalan orang-orang yang Engkau telah kurniakan nikmat kepada mereka, bukan (jalan) orang-orang yang Engkau telah murkai, dan bukan pula (jalan) orang-orang yang sesat.',
  'The path of those You have blessed, Not of those who have earned Your anger, nor those who have gone as

In [16]:
import json

with open('tanzil-ms-en.json', 'w') as fopen:
    json.dump(results, fopen)

In [17]:
!rm -rf Tanzil
!rm -rf ms.zip en.zip
!rm -rf ms.zip en.zip en-ms.xml.gz en-ms.xml